## joda_01_kehitysympäristö
Jupyter Notebook (ver. 5.7.8) ympäristö Anaconda (ver. 1.9.7) jakeluna omalle PC:lle asennettuna. <br>
Asennus onnistuu https://www.anaconda.com/distribution/ sivuilta valitsemalla oikea käyttöjärjestelmä ja haluttu Python versio.<br>

Editorina Microsoft Visual Studio Code (ver. 1.40.1) PC:lle asennettuna. <br>
Asennus onnistuu https://code.visualstudio.com/download sivuilta valitsemalla oikea käyttöjärjestelmä. <br>

GitHub toimii julkaisualustana sivulla https://github.com/mvehmane/airbnb_munich <br>
GitHub tili on tullut joskus aikaisemmin aktivoitua vaikka en sitä olekaan varsinaisesti käyttänyt. <br>
GitHub on käytettävissä sekä web että desktop ympäristössä. <br>
Asennan myös desktop version harjoittelua varten. <br>
Desktop asennus https://desktop.github.com/ <br>

Näillä työkaluilla yritetään pärjätä ja tarvittaessa asennetaan lisää harjoituksen tekemiseksi.


### kirjastojen haku
asennetaan kirjastot yhdessä solussa: <br>
pandas - dataframes <br>
numpy - math <br>
sklearn - machine learning <br>
scipy - numerical plotting<br>
matplotlib - numerical plotting <br>
seaborn - data visualization <br>
plotly - data analytics and visualization <br>
dash - analytical web applications <br>








In [2]:
#kirjastojen asennus
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn import ensemble
from sklearn import linear_model
from sklearn.model_selection import learning_curve,GridSearchCV
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
import matplotlib.pyplot as plt
from collections import Counter
import scipy
import matplotlib
import seaborn as sns


## joda_02_datan_kerääminen
Käytetään insideairbnb sivuston dataa kohdekaupunkina Munich, käytettävissä on kolme datapakettia. <br>
Inside Airbnb sivusto ei kerro tarkemmin datan ominaisuuksista, joten on itse arvioitava eri tietojen hyödyllisyyttä ja käyttökelpoisuutta. <br>
Haetaan kaikki kolme datapakettia: <br>
1. Detailed Listings data for Munich <br>
2. Detailed Calendar Data for listings in Munich <br>
3. Detailed Review Data for listings in Munich <br>




In [13]:
listings_file = 'http://data.insideairbnb.com/germany/bv/munich/2019-09-24/data/listings.csv.gz'

# read listings file into a dataframe
df_listings = pd.read_csv(listings_file, compression='gzip')
df_listings.to_csv('listings.csv')
# tutkitaan datasettiä sarakenimien avulla
pd.options.display.max_rows=110
pd.Series(df_listings.columns)

0                                                id
1                                       listing_url
2                                         scrape_id
3                                      last_scraped
4                                              name
5                                           summary
6                                             space
7                                       description
8                               experiences_offered
9                             neighborhood_overview
10                                            notes
11                                          transit
12                                           access
13                                      interaction
14                                      house_rules
15                                    thumbnail_url
16                                       medium_url
17                                      picture_url
18                                   xl_picture_url
19          

In [7]:
# read calendar file into a dataframe
calendar_file = 'http://data.insideairbnb.com/germany/bv/munich/2019-09-24/data/calendar.csv.gz'
df_calendar = pd.read_csv(calendar_file, compression='gzip')
df_calendar.to_csv('calendar.csv')
# sarakkeiden listaus
df_calendar.columns

Index(['listing_id', 'date', 'available', 'price', 'adjusted_price',
       'minimum_nights', 'maximum_nights'],
      dtype='object')

In [9]:
# read reviews file into a dataframe
reviews_file = 'http://data.insideairbnb.com/germany/bv/munich/2019-09-24/data/reviews.csv.gz'
df_reviews = pd.read_csv(reviews_file, compression='gzip')
df_reviews.to_csv('reviews.csv')
# sarakkeiden listaus
df_reviews.columns

Index(['listing_id', 'id', 'date', 'reviewer_id', 'reviewer_name', 'comments'], dtype='object')

In [14]:
df_reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,36720,12723661,2014-05-09,11840468,Mikhail,I would like to thank Gabriella as her apartme...
1,36720,13147830,2014-05-20,5466213,Kim,Gabriela's place was absolutely fantastic. It...
2,36720,16302574,2014-07-25,2062882,Juan R.,"Quiet place, open to a courtyard, with all the..."
3,36720,16428874,2014-07-27,1225618,David,The best Airbnb expeierence I've had. The apar...
4,36720,19478358,2014-09-13,13977301,Cal,"All first rate ! Beautiful apartment, comforta..."


In [16]:
# yhdistetään df_listings ja df.reviews datasetit kohteen id:llä

df_listings_reviews = df_listings.merge(df_reviews, left_on='id', right_on='listing_id')

In [19]:
df_listings_reviews.head(3)


,id_x,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,listing_id,id_y,date,reviewer_id,reviewer_name,comments
0,36720,https://www.airbnb.com/rooms/36720,20190924033931,2019-09-24,"Beautiful 2 rooms flat, Glockenbach","Very quiet, yet centrally located 2 rooms apt....","2-Zimmer Altbauwohung Beste Lage, zentral, im ...","Very quiet, yet centrally located 2 rooms apt....",none,Glockenbachviertel area has the hippest restau...,...,1,0,0,0.38,36720,12723661,2014-05-09,11840468,Mikhail,I would like to thank Gabriella as her apartme...
1,36720,https://www.airbnb.com/rooms/36720,20190924033931,2019-09-24,"Beautiful 2 rooms flat, Glockenbach","Very quiet, yet centrally located 2 rooms apt....","2-Zimmer Altbauwohung Beste Lage, zentral, im ...","Very quiet, yet centrally located 2 rooms apt....",none,Glockenbachviertel area has the hippest restau...,...,1,0,0,0.38,36720,13147830,2014-05-20,5466213,Kim,Gabriela's place was absolutely fantastic. It...
2,36720,https://www.airbnb.com/rooms/36720,20190924033931,2019-09-24,"Beautiful 2 rooms flat, Glockenbach","Very quiet, yet centrally located 2 rooms apt....","2-Zimmer Altbauwohung Beste Lage, zentral, im ...","Very quiet, yet centrally located 2 rooms apt....",none,Glockenbachviertel area has the hippest restau...,...,1,0,0,0.38,36720,16302574,2014-07-25,2062882,Juan R.,"Quiet place, open to a courtyard, with all the..."
